#  Chapter2 機械学習の流れ

# 2.1 機械学習の流れ（P.26）  
* 教師あり学習で行うことの流れ  

    1. データ収集
    1. データクレンジング（データの精度を高める） 
    1. 機械学習手法でデータを学習
    1. テストデータで性能をテスト
    1. 機械学習モデルをWebなどに実装


* **機械学習を行うといっても事前の準備や結果の考察が必要**
    * 多くの場合データ収集・データクレンジングに時間がかかる


* データの学習  
    * 「Iris（あやめ）」
        * 機械学習でよく使われるサンプルデータセット
            * [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)からダウンロード可能
                * 
            * がく片（sepal）と花びら（petal）の長さと花弁とその品種
    * 品種（setosa, versicolor）を分類する
        * 適当に線を引く（分類する）
        * その線が妥当な位置にあるかを計算で求める
        * 線の位置を改善するほうに修正する
        * 適切に点を分類できる位置に線を引けたら終了
    * モデル
        * 学習の結果、コンピュータ自身が自分で答えを見つけ、データのパターンから作られた基準

## 2.2 学習データの使い方（P.30）
* 「教師あり学習」…扱うデータを「トレーニングデータ」と「テストデータ」に分けて用いる
    * 機械学習は「未知のデータを予測する」ことが目的
    * モデル評価には、学習には使われていないデータ（テストデータ）を用いる
        * 「MNIST」では60,000枚がトレーニングデータ、10,000枚がテストデータ
        * 多くの場合は全体の20%ほどをテストデータに用いる
* データ分割方法  
    * ホールドアウト法
    * k-分割交差検証（k-クロスバリデーション）法
    

* ホールドアウト法
    * scikit-learnに用意された関数train_test_split()が使える
        * [**`train_test_split(*arrays, **options)`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)  
        * test_size=XXXで、データ全体からテストデータとして選びたい割合を0～1までの数値で指定
        * random_state=0は実験の再現性を担保するために指定


* k-分割交差検証（k-クロスバリデーション）
    * 非復元抽出を用いて、トレーニングデータセットをk分割し、そのうちのk-1個のデータを学習データセットとして用い、  
    残りの1個をモデルのテストに用いる手法
    * 結果として、k個のモデルとそのモデルに対する性能評価がk個得られる  
        * k個の性能評価の平均をとり、平均性能を算出する
        * より安定した正確なモデル評価ができる
        * ホールドアウト法よりもk倍の演算が必要
    * 一般的に用いられるkの値は5～10
        * データセットが大きい場合はkの値を増やすことでよい結果が得られる場合が多い
        * データセットがかなり小さい場合、一個抜き（Leave-One-Out:LOO）交差検証が推奨
            * 例えばデータセットが50～100行以下
    * scikit-learnに用意された関数model_selection.cross_val_score()が使える
        * [**`model_selection.cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv=’warn’, n_jobs=None, verbose=0, fit_params=None, pre_dispatch=‘2*n_jobs’, error_score=’raise-deprecating’)`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
        * バージョン0.20以降、cross_validationモジュールからmodel_selectionモジュールに移動

### リスト 2.1（P.33）

In [1]:
# コードの実行に必要なモジュールを読み込みます
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Irisというデータセットを読み込みます
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 「X_train, X_test, y_train, y_test」にデータを格納します
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# トレーニングデータとテストデータのサイズを確認します
print ("X_train :", X_train.shape)
print ("y_train :", y_train.shape)
print ("X_test :", X_test.shape)
print ("y_test :", y_test.shape)

X_train : (120, 4)
y_train : (120,)
X_test : (30, 4)
y_test : (30,)


### リスト 2.3（P.37）

In [2]:
# コードの実行に必要なモジュールを読み込みます
##from sklearn import svm, datasets, cross_validation
from sklearn import svm, datasets, model_selection

# 「Iris」というデータセットを読み込みます
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 機械学習アルゴリズムSVMを使用します
svc = svm.SVC(C=1, kernel="rbf", gamma=0.001)

# 交差検証法を用いてスコアを求めます
# 内部では、X、yがそれぞれ「X_train, X_test, y_train, y_test」の様に分割され処理されます
scores = model_selection.cross_val_score(svc, X, y, cv=5)

# トレーニングデータとテストデータのサイズを確認します
print (scores)
print ("平均スコア :", scores.mean())

[0.86666667 0.96666667 0.83333333 0.96666667 0.93333333]
平均スコア : 0.9133333333333334


## 2.3 過学習（P.38）
* 過学習（オーバーフィッティング）
    * コンピュータが学習しすぎてしまい、正しい基準が構築されないこと
    * 過学習を起こしているモデル：バリアンスが高い
* 過学習の回避  
    * 深層学習（ディープラーニング）の場合、「Dropout（ドロップアウト）」という手法を用いる
        * 学習時にランダムに一部のニューロンを消し去る
    * 正則化
        * 偏りがあるデータの影響をなくす（消す）方法
* 学習不足
    * コンピュータがデータを学習できていない状態
    * 学習不足を起こしているモデル：バイアスが高い

## 2.4 アンサンブル学習（P.41）  
* アンサンブル学習
    * 複数のモデルに学習させることにより、データの一般化を獲得しようとする試み
        * バギング
            * 複数のモデルを同時に学習させ、予測結果の平均をとることで予測結果の汎化を試みる
        * ブースティング
            * モデルの予測結果に対するモデルを作成し、汎化性能を高める手法

# ほげ要望
第２章の添削問題をやってくれたら工藤が喜びます

# エクストラほげ問題②
## Pythonで学ぶ線形代数
## ベクトルの大きさと単位ベクトル
大学に進学をした理系学生が１年生の最初に習うのが線形代数です。その知識を使い今習っているPythonの技術向上を目指しましょう。   
※ヒントのようなもの  
①https://oguemon.com/topic/study/linear-algebra/  
②https://mathtrain.jp/category/senkei  
③https://python.atelierkobato.com/linear/  
④https://examist.jp/category/mathematics/  
⑤http://www.geisya.or.jp/~mwm48961/koukou/index_m.htm  
  
# もんだいだよん(^ω^)
## ベクトルの大きさ
### ①ユークリッドノルム
PythonのライブラリNumPyを用いて解きなさい  
ベクトル$\vec{a}$のユークリッドノルムを求めなさい
  
$
  \vec{a} = \left(
    \begin{array}{ccc}
      1 \\
      2 \\
    \end{array}
  \right)
$
### ②scipy.linalg.norm()
PythonのライブラリNumPyを用いて解きなさい  
ベクトル$\vec{a}$のユークリッドノルムをscipy.linalg.norm()を用いて求めなさい
  
$
  \vec{a} = \left(
    \begin{array}{ccc}
      1 \\
      2 \\
    \end{array}
  \right)
$
## 単位ベクトル
### ③単位ベクトル
PythonのライブラリNumPyを用いて解きなさい  
ベクトル$\vec{a}$の単位ベクトルを求めなさい
  
$
  \vec{a} = \left(
    \begin{array}{ccc}
      1 \\
      2 \\
    \end{array}
  \right)
$
# ヒントのようなもの
以下の流れでプログラムしたらいいと思います  

``` python
#ライブラリのインポート

#ベクトルやスカラー定義

#計算処理

#結果のプリント

```